In [1]:
import pandas as pd
import io
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
links_df= pd.read_csv('match_links.csv')
links_df['link']

0     https://www.espncricinfo.com/series/west-indie...
1     https://www.espncricinfo.com/series/west-indie...
2     https://www.espncricinfo.com/series/west-indie...
3     https://www.espncricinfo.com/series/bangladesh...
4     https://www.espncricinfo.com/series/bangladesh...
                            ...                        
69    https://www.espncricinfo.com/series/icc-champi...
70    https://www.espncricinfo.com/series/icc-champi...
71    https://www.espncricinfo.com/series/bangladesh...
72    https://www.espncricinfo.com/series/bangladesh...
73    https://www.espncricinfo.com/series/bangladesh...
Name: link, Length: 74, dtype: object

In [5]:
driver.get(links_df.loc[1,'link'])

In [120]:
driver.quit()

In [4]:
driver = webdriver.Chrome()
driver.get("https://www.espncricinfo.com/team/bangladesh-25/match-schedule-fixtures-and-results?format=2")
driver.implicitly_wait(16)
popup_container_csss = "#wzrk_wrapper > div.wzrk-alert.wiz-show-animate"
popup_container = driver.find_element(By.CSS_SELECTOR, popup_container_csss)
popup_buttons = popup_container.find_elements(By.CSS_SELECTOR, 'button')
for b in popup_buttons:
    if 'not now' in b.get_attribute('innerHTML').lower(): b.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#wzrk_wrapper > div.wzrk-alert.wiz-show-animate"}
  (Session info: chrome=139.0.7258.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff70d9b3d85+79397]
	GetHandleVerifier [0x0x7ff70d9b3de0+79488]
	(No symbol) [0x0x7ff70d75c0fa]
	(No symbol) [0x0x7ff70d7b2fd6]
	(No symbol) [0x0x7ff70d7b328c]
	(No symbol) [0x0x7ff70d806537]
	(No symbol) [0x0x7ff70d7db1df]
	(No symbol) [0x0x7ff70d803344]
	(No symbol) [0x0x7ff70d7daf73]
	(No symbol) [0x0x7ff70d7a41b1]
	(No symbol) [0x0x7ff70d7a4f43]
	GetHandleVerifier [0x0x7ff70dc7e1ed+3005069]
	GetHandleVerifier [0x0x7ff70dc7831d+2980797]
	GetHandleVerifier [0x0x7ff70dc97e0d+3110573]
	GetHandleVerifier [0x0x7ff70d9cd6de+184190]
	GetHandleVerifier [0x0x7ff70d9d516f+215567]
	GetHandleVerifier [0x0x7ff70d9bc974+1

In [35]:
table = driver.find_elements(By.CSS_SELECTOR, 'table')[-1].get_attribute('outerHTML')

pd.read_html(io.StringIO(table))[0].transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,"Shere Bangla National Stadium, Mirpur",Toss,Series,Season,Player Of The Match,Series result,Match number,Match days,ODI debut,Umpires,TV Umpire,Reserve Umpire,Match Referee,Points
1,"Shere Bangla National Stadium, Mirpur","West Indies, elected to bat first",West Indies tour of BangladeshICC Men's Cricke...,2020/21,Mehidy Hasan Miraz,Bangladesh led the 3-match series 2-0,ODI no. 4272,22 January 2021 - day (50-over match),Kjorn Ottley,Gazi SohelSharfuddoula,Masudur Rahman,Tanvir Ahmed,Neeyamur Rashid,"Bangladesh 10, West Indies 0"


In [6]:
match_stats_records = []
undecided_match_links = []
driver = webdriver.Chrome()
for i, link in enumerate(links_df['link'][:10]):
    driver.get(link)
    time.sleep(.5)
    summary = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > p > span > span')
    if 'won' in summary.text:
        title_text = driver.find_element(By.CSS_SELECTOR,r'#main-container > div:nth-child(2) > div > div > div > div.ds-grow.ds-flex.ds-items-center > div > h1').text
        short_description = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > div.ds-flex.ds-justify-between.ds-items-center > div.ds-truncate > span.ds-flex.ds-items-center > div').text
        ltable = pd.read_html(io.StringIO(driver.find_elements(By.CSS_SELECTOR, 'table')[-1].get_attribute('outerHTML')))[0]
        ltable = ltable.rename(index=ltable.loc[:,0])

        venue = ltable.iloc[0,1]
        toss_text = ltable.loc['Toss', 1]
        season_text = ltable.loc['Season', 1]
        player_of_the_match = ltable.loc['Player Of The Match',1]
        series_result = ltable.loc['Series result', 1]
        match_days = ltable.loc['Match days', 1]

        record = {
                'title' : title_text,
                'short_description': short_description,
                'result' : summary.text,
                'venue' : venue,
                'toss_text': toss_text,
                'season_text': season_text,
                'player_of_the_match': player_of_the_match,
                'series_result': series_result,
                'match_days': match_days,
                'scorecard_url': link
            }
        match_stats_records.append(
            record
        )
    else:
        undecided_match_links.append({
            'iteration': i,
            'link': link
        })
        

driver.quit()


In [7]:
match_stats_records[0]

{'title': 'BAN vs WI',
 'short_description': '1st ODI, Mirpur, January 20, 2021, West Indies tour of Bangladesh',
 'result': 'Bangladesh won by 6 wickets (with 97 balls remaining)',
 'venue': 'Shere Bangla National Stadium, Mirpur',
 'toss_text': 'Bangladesh, elected to field first',
 'season_text': '2020/21',
 'player_of_the_match': 'Shakib Al Hasan',
 'series_result': 'Bangladesh led the 3-match series 1-0',
 'match_days': '20 January 2021 - day (50-over match)',
 'scorecard_url': 'https://www.espncricinfo.com/series/west-indies-in-bdesh-2020-21-1244017/bangladesh-vs-west-indies-1st-odi-1244022/full-scorecard'}

In [8]:
for match_stats in match_stats_records:
    home_team, away_team = match_stats['title'].split(' vs ')
    print(home_team,
        away_team,
        match_stats['venue'])

BAN WI Shere Bangla National Stadium, Mirpur
BAN WI Shere Bangla National Stadium, Mirpur
BAN WI Zahur Ahmed Chowdhury Stadium, Chattogram
NZ BAN University Oval, Dunedin
NZ BAN Hagley Oval, Christchurch
NZ BAN Basin Reserve, Wellington
BAN SL Shere Bangla National Stadium, Mirpur
BAN SL Shere Bangla National Stadium, Mirpur
BAN SL Shere Bangla National Stadium, Mirpur
ZIM BAN Harare Sports Club


In [3]:
url = links_df['link'].iloc[1]

driver.get(url)

NameError: name 'driver' is not defined

In [48]:
links_df['link'].iloc[1]

'https://www.espncricinfo.com/series/west-indies-in-bdesh-2020-21-1244017/bangladesh-vs-west-indies-2nd-odi-1244023/full-scorecard'

In [146]:
tables = driver.find_elements(By.CSS_SELECTOR, 'table')

ltable = pd.read_html(io.StringIO(tables[-1].get_attribute('outerHTML')))[0]

ltable = ltable.rename(index=ltable.loc[:,0])
venue = ltable.iloc[0,1]
toss_text = ltable.loc['Toss', 1]
season_text = ltable.loc['Season', 1]
player_of_the_match = ltable.loc['Player Of The Match',1]
series_result = ltable.loc['Series result', 1]
match_days = ltable.loc['Match days', 1]

bd_won_toss = True if 'Bangladesh' in toss_text else False
if bd_won_toss and 'bat' in toss_text: bd_batting_first = True
elif not bd_won_toss and 'bat' in toss_text: bd_batting_first = False
else: bd_batting_first = None 
if bd_batting_first: 
    bd_batting_table_i = 0
    bd_bowling_table_i = 3
elif not bd_batting_first:
    bd_batting_table_i = 2
    bd_bowling_table_i = 1

bd_batting_table = pd.read_html(
    io.StringIO(
        tables[bd_batting_table_i].get_attribute('outerHTML')
    )
)[0]
bd_bowling_table = pd.read_html(
    io.StringIO(
        tables[bd_bowling_table_i].get_attribute('outerHTML')
    )
)[0]

#dn_bat_list = (bd_batting_table[bd_batting_table.loc[:,'Batting'].str.contains('Did not bat')].iloc[0,0]).split(':')[1].split(',\xa0') 
bd_batting_table = bd_batting_table[bd_batting_table['Batting']!= bd_batting_table['R']]
bd_batting_table = bd_batting_table[['Batting', 'R', 'B', '4s', '6s', 'SR']].dropna(how='all')
bd_batting_table= bd_batting_table[~bd_batting_table['Batting'].str.casefold().str.contains('total|extras|did not bat|fall of wickets')]
bat_list = bd_batting_table.iloc[:,0]
bat_list

### Finding captain:
if bat_list[bat_list.str.contains('(c)')].size != 0:
    print(bat_list)
    captain = bat_list[bat_list.str.contains('(c)')].item().replace('\xa0','').replace('(c)','').strip()
else:
    #PRINT LINK
    captain = "Did not bat"
captain
bd_batting_table.to_csv('CHANGE_batting_table.csv')
bd_bowling_table = bd_bowling_table[bd_bowling_table['Bowling'] != bd_bowling_table['O']]
bd_bowling_table.to_csv('CHANGE_bowling_table.csv')

record = {
    #'title' : title_text,
    #'short_description': short_description,
    #'result' : summary.text,
    'venue' : venue,
    'toss_text': toss_text,
    'season_text': season_text,
    'player_of_the_match': player_of_the_match,
    'series_result': series_result,
    'match_days': match_days,
    'bd_won_toss': bd_won_toss,
    'bd_batting_first': bd_batting_first,
    'bd_captain': captain

    #'scorecard_url': link
}


0      Martin Guptill
2      Henry Nicholls
4        Devon Conway
6          Will Young
8     Tom Latham (c)†
10      James Neesham
12     Daryl Mitchell
Name: Batting, dtype: object


C:\Users\nukag\AppData\Local\Temp\ipykernel_9104\724714402.py:43: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if bat_list[bat_list.str.contains('(c)')].size != 0:
C:\Users\nukag\AppData\Local\Temp\ipykernel_9104\724714402.py:45: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  captain = bat_list[bat_list.str.contains('(c)')].item().replace('\xa0','').replace('(c)','').strip()


ValueError: can only convert an array of size 1 to a Python scalar

In [3]:
match_stats_records = []
undecided_match_links = []
driver = webdriver.Chrome()
for i, link in enumerate(links_df['link']):
    print('testing i:', i)
    driver.get(link)
    time.sleep(.5)
    summary = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > p > span > span')
    if 'won' in summary.text:
        title_text = driver.find_element(By.CSS_SELECTOR,r'#main-container > div:nth-child(2) > div > div > div > div.ds-grow.ds-flex.ds-items-center > div > h1').text
        short_description = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > div.ds-flex.ds-justify-between.ds-items-center > div.ds-truncate > span.ds-flex.ds-items-center > div').text
        tables = driver.find_elements(By.CSS_SELECTOR, 'table')

        ltable = pd.read_html(io.StringIO(tables[-1].get_attribute('outerHTML')))[0]

        ltable = ltable.rename(index=ltable.loc[:,0])
        venue = ltable.iloc[0,1]
        toss_text = ltable.loc['Toss', 1]
        season_text = ltable.loc['Season', 1]
        player_of_the_match = ltable.loc['Player Of The Match',1]
        match_days = ltable.loc['Match days', 1]
        

        bd_won_toss = True if 'Bangladesh' in toss_text else False
        if bd_won_toss and 'bat' in toss_text: bd_batting_first = True
        elif not bd_won_toss and 'bat' in toss_text: bd_batting_first = False
        else: bd_batting_first = None

        record = {
            'title' : title_text,
            'short_description': short_description,
            'result' : summary.text,
            'venue' : venue,
            'toss_text': toss_text,
            'season_text': season_text,
            'player_of_the_match': player_of_the_match,
            'match_days': match_days,
            'bd_won_toss': bd_won_toss,
            'bd_batting_first': bd_batting_first,
            'scorecard_url': link
        }
        if 'Series result' in ltable.columns:
            series_result = ltable.loc['Series result', 1]
            record['series_result'] = series_result
        else: record['series_result'] = None
        match_stats_records.append(
            record
        )
    else:
        undecided_match_links.append({
            'iteration': i,
            'link': link
        })
        

driver.quit()


testing i: 0
testing i: 1
testing i: 2
testing i: 3
testing i: 4
testing i: 5
testing i: 6
testing i: 7
testing i: 8
testing i: 9
testing i: 10
testing i: 11
testing i: 12
testing i: 13
testing i: 14
testing i: 15
testing i: 16
testing i: 17
testing i: 18
testing i: 19
testing i: 20
testing i: 21
testing i: 22
testing i: 23
testing i: 24
testing i: 25
testing i: 26
testing i: 27
testing i: 28
testing i: 29
testing i: 30
testing i: 31
testing i: 32
testing i: 33
testing i: 34
testing i: 35
testing i: 36
testing i: 37
testing i: 38
testing i: 39
testing i: 40
testing i: 41
testing i: 42
testing i: 43
testing i: 44
testing i: 45
testing i: 46
testing i: 47
testing i: 48
testing i: 49
testing i: 50
testing i: 51
testing i: 52
testing i: 53
testing i: 54
testing i: 55
testing i: 56
testing i: 57
testing i: 58
testing i: 59
testing i: 60
testing i: 61
testing i: 62
testing i: 63
testing i: 64
testing i: 65
testing i: 66
testing i: 67
testing i: 68
testing i: 69
testing i: 70
testing i: 71
te

In [28]:
ltable

,0,1
Pallekele International Cricket Stadium,Pallekele International Cricket Stadium,Pallekele International Cricket Stadium
Toss,Toss,"Bangladesh, elected to bat first"
Series,Series,Asia Cup
Season,Season,2023
Player Of The Match,Player Of The Match,Matheesha Pathirana
Match number,Match number,ODI no. 4629
Hours of play (local time),Hours of play (local time),"15.00 start, First Session 15.00-18.30, Interv..."
Match days,Match days,31 August 2023 - daynight (50-over match)
ODI debut,ODI debut,Tanzid Hasan
Umpires,Umpires,Jayaraman MadanagopalDRSPaul WilsonDRS


In [17]:
match_stats_records

[{'title': 'BAN vs WI',
  'short_description': '1st ODI, Mirpur, January 20, 2021, West Indies tour of Bangladesh',
  'result': 'Bangladesh won by 6 wickets (with 97 balls remaining)',
  'venue': 'Shere Bangla National Stadium, Mirpur',
  'toss_text': 'Bangladesh, elected to field first',
  'season_text': '2020/21',
  'player_of_the_match': 'Shakib Al Hasan',
  'match_days': '20 January 2021 - day (50-over match)',
  'bd_won_toss': True,
  'bd_batting_first': None,
  'scorecard_url': 'https://www.espncricinfo.com/series/west-indies-in-bdesh-2020-21-1244017/bangladesh-vs-west-indies-1st-odi-1244022/full-scorecard',
  'series_result': None,
  'home': 'BAN',
  'away': 'WI',
  'time': Timestamp('2021-01-20 00:00:00'),
  'bd_won_match': True,
  'match_was_draw': False},
 {'title': 'BAN vs WI',
  'short_description': '2nd ODI, Mirpur, January 22, 2021, West Indies tour of Bangladesh',
  'result': 'Bangladesh won by 7 wickets (with 100 balls remaining)',
  'venue': 'Shere Bangla National Stad

In [ ]:
wrangled = []
for mr in match_stats_records:
    home, away = mr['title'].split(' vs ')
    time_str = mr['match_days'].split(' - ')[0]
    datetime = pd.to_datetime(time_str, dayfirst=True)
    mr['home'] = home
    mr['away'] = away
    mr['time'] = datetime
    mr['bd_won_match'] = True if 'Bangladesh won' in mr['result'] else False
    mr['match_was_draw'] = True if 'Draw' in mr['result'] else False
    mr['bd_won_series'] = True if 'Bangladesh' in mr['series_result'] else False
    wrangled.append(mr)
df = pd.DataFrame(wrangled)
df = df.drop(['title', 'short_description', 'toss_text'], axis=1)

df.to_csv('match_stats.csv')

TypeError: argument of type 'NoneType' is not iterable

In [19]:
mr = match_stats_records[1]
home, away = mr['title'].split(' vs ')
time_str = mr['match_days'].split(' - ')[0]
time = pd.to_datetime(time_str, dayfirst=True)
time

Timestamp('2021-01-22 00:00:00')

In [ ]:
match_stats_records = []
undecided_match_links = []
driver = webdriver.Chrome()
for i, link in enumerate(links_df['link']):
    print('testing i:', i)
    driver.get(link)
    time.sleep(.5)
    summary = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > p > span > span')
    if 'won' in summary.text:
        title_text = driver.find_element(By.CSS_SELECTOR,r'#main-container > div:nth-child(2) > div > div > div > div.ds-grow.ds-flex.ds-items-center > div > h1').text
        short_description = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > div.ds-flex.ds-justify-between.ds-items-center > div.ds-truncate > span.ds-flex.ds-items-center > div').text
        tables = driver.find_elements(By.CSS_SELECTOR, 'table')

        ltable = pd.read_html(io.StringIO(tables[-1].get_attribute('outerHTML')))[0]

        ltable = ltable.rename(index=ltable.loc[:,0])
        venue = ltable.iloc[0,1]
        toss_text = ltable.loc['Toss', 1]
        season_text = ltable.loc['Season', 1]
        player_of_the_match = ltable.loc['Player Of The Match',1]
        match_days = ltable.loc['Match days', 1]
        

        bd_won_toss = True if 'Bangladesh' in toss_text else False
        if bd_won_toss and 'bat' in toss_text: bd_batting_first = True
        elif not bd_won_toss and 'bat' in toss_text: bd_batting_first = False
        else: bd_batting_first = None

        record = {
            'title' : title_text,
            'short_description': short_description,
            'result' : summary.text,
            'venue' : venue,
            'toss_text': toss_text,
            'season_text': season_text,
            'player_of_the_match': player_of_the_match,
            'match_days': match_days,
            'bd_won_toss': bd_won_toss,
            'bd_batting_first': bd_batting_first,
            'scorecard_url': link
        }
        if 'Series result' in ltable.columns:
            series_result = ltable.loc['Series result', 1]
            record['series_result'] = series_result
        else: record['series_result'] = None
        match_stats_records.append(
            record
        )
    else:
        undecided_match_links.append({
            'iteration': i,
            'link': link
        })
        

driver.quit()


In [30]:
driver.quit()
match_stats_records = []
undecided_match_links = []
driver = webdriver.Chrome()
for i, link in enumerate(links_df['link']):
    driver.get(link)
    time.sleep(.5)
    summary = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > p > span > span')
    if 'won' in summary.text:
        title_text = driver.find_element(By.CSS_SELECTOR,r'#main-container > div:nth-child(2) > div > div > div > div.ds-grow.ds-flex.ds-items-center > div > h1').text
        short_description = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(1) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden > div > div:nth-child(1) > div > div > div > div > div.ds-flex.ds-justify-between.ds-items-center > div.ds-truncate > span.ds-flex.ds-items-center > div').text
        tables = driver.find_elements(By.TAG_NAME, 'table')
        for j, t in enumerate([x.get_attribute('outerHTML') for x in tables]):
            with open(f'./raw_tables/match_{i}_table_{j}.html', 'w') as f:
                f.write(t)
        first_table_name = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(2) > div:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden.ds-border-line.ds-mb-2.ds-border > div.ds-flex.ds-px-4.ds-border-b.ds-border-line.ds-py-3.ds-bg-ui-fill-translucent-hover > div > span > span.ds-text-title-xs.ds-font-bold.ds-capitalize')
        match_stats_records.append({
            'title': title_text,
            'summary_text': summary.text,
            'short_description': short_description,
            'first_table_name': first_table_name.text,
            'link': link,
        })

        
    else:
        undecided_match_links.append({
            'index': i,
            'link': link
        })
        

driver.quit()


In [35]:
pd.DataFrame(match_stats_records).to_csv('raw_match_stats.csv')

In [36]:
pd.DataFrame(undecided_match_links).to_csv('raw_undecided_match_stats.csv')

In [13]:
undecided_match_links

[{'iteration': 1,
  'link': 'https://www.espncricinfo.com/series/ireland-in-bangladesh-2022-23-1355710/bangladesh-vs-ireland-2nd-odi-1355718/full-scorecard'},
 {'iteration': 3,
  'link': 'https://www.espncricinfo.com/series/ireland-v-bangladesh-2023-1364121/bangladesh-vs-ireland-1st-odi-1364124/full-scorecard'}]

In [15]:
links_df['link'][0]

'https://www.espncricinfo.com/series/west-indies-in-bdesh-2020-21-1244017/bangladesh-vs-west-indies-1st-odi-1244022/full-scorecard'

In [14]:
driver= webdriver.Chrome()

In [16]:
driver.get(links_df['link'][0])

In [9]:
first_table_name = driver.find_element(By.CSS_SELECTOR, r'#main-container > div.lg\:ds-container.lg\:ds-mx-auto.lg\:ds-px-5.lg\:ds-pt-4 > div > div > div:nth-child(2) > div:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div.ds-w-full.ds-bg-fill-content-prime.ds-overflow-hidden.ds-border-line.ds-mb-2.ds-border > div.ds-flex.ds-px-4.ds-border-b.ds-border-line.ds-py-3.ds-bg-ui-fill-translucent-hover > div > span > span.ds-text-title-xs.ds-font-bold.ds-capitalize')
first_table_name.text

'West Indies'

In [27]:
driver.quit()

In [23]:
tables = driver.find_elements(By.TAG_NAME, 'table')
for i, t in enumerate([x.get_attribute('outerHTML') for x in tables]):
    with open(f'./raw_tables/{i}.html', 'w') as f:
        f.write(t)

In [25]:
pd.read_html('./raw_tables/0.html')

[                                              Batting  \
 0                                        Sunil Ambris   
 1                             lbw b Mustafizur Rahman   
 2                                   Joshua Da Silva    
 3                    c Litton Das b Mustafizur Rahman   
 4                                      Andre McCarthy   
 5                                   b Shakib Al Hasan   
 6                                  Jason Mohammed (c)   
 7               st Mushfiqur Rahim b Shakib Al Hasan   
 8                                         Kyle Mayers   
 9                   c Litton Das b Mehidy Hasan Miraz   
 10                                     Nkrumah Bonner   
 11                              lbw b Shakib Al Hasan   
 12                                      Rovman Powell   
 13                  c Mushfiqur Rahim b Hasan Mahmud   
 14                                      Raymon Reifer   
 15                                 lbw b Hasan Mahmud   
 16           